In [8]:
from tkinter import *
from tkinter import filedialog
import os
import Tensile_Table

#def sel():
#    if glass_var=='Break Properties':
#        print('BREAK')
   
def browse_folder():
    global filename
    filename = filedialog.askdirectory()
    folder_label.configure(text = "Files will be saved to: " + str(filename), wraplength = 250)
    print(filename)
    return filename
    os.chdir(str(filename))

filenames = []
entries = []
column_names = []
def select_files():
    files = filedialog.askopenfilenames()
    for item in files:
        lb1.insert(END, item)
        filenames.append(item)
    column_entry_label = Label(root, text = 'Please enter a corresponding column name for each file').grid(row=6,column=1)
    entry_frame = Frame(root,borderwidth=1, relief="solid")
    entry_frame.grid(row=7,column=1)
    for i in range(len(list(filenames))):
        en = Entry(entry_frame)
        en.grid(row=i+1)
        entries.append(en)
 

def make_table():
    for entry in entries:
        column_names.append(entry.get())
    import pandas as pd
    import os
    import matplotlib.pyplot as plt
    import numpy as np
    import docx
    from docx.shared import Inches
    from docx.shared import Pt
    from docx.enum.text import WD_ALIGN_PARAGRAPH
    
    #Set working directory to desired outpur folder
    os.chdir(str(filename))
    
    #list of files to work on. These must be generated from the other pyhton programs I made
    files = filenames
    names = column_names
    
    
    #pulls modulus, stress, strain values from the reports above and creates plot of all "sample 1's"
    moduli = []
    moduli_dev = []
    stress = []
    stress_dev = []
    strain = []
    strain_dev = []
    table_title = table_title_entry.get()
    file_title = file_title_entry.get()
    
    #strain_indexer = ''
    if glass_var.get()=='Break Properties':
        strain_indexer = 'Strain at Break (%)'
        stress_indexer = 'Stress at Break (MPa)'
    else:
        strain_indexer = 'Strain at Yield (%)'
        stress_indexer = 'Stress at Yield (MPa)'
        
    plt.figure(figsize = (10,5))
    for i in range(len(list(files))):
     df = pd.read_excel(files[i], sheet_name = 'Report')
     mod = df.loc['Modulus (MPa)', 'AVG']
     mod_dev = df.loc['Modulus (MPa)', 'DEV'].round(1)
     moduli.append(mod)
     moduli_dev.append(mod_dev)
     stress_ab = df.loc[stress_indexer, 'AVG'].round(1)
     stress_ab_dev = df.loc[stress_indexer, 'DEV'].round(1)
     stress.append(stress_ab)
     stress_dev.append(stress_ab_dev)
     strain_ab = df.loc[strain_indexer, 'AVG'].round(2)
     strain_ab_dev = df.loc[strain_indexer, 'DEV'].round(1)
     strain.append(strain_ab)
     strain_dev.append(strain_ab_dev)

     max_stress = max(stress)
     stress_axis_limit = max_stress*1.1
     max_strain = max(strain)
     strain_axis_limit = max_strain*1.1

     plotdf = pd.read_excel(files[i], sheet_name = 'Sample1RawData', Skiprows=5)
     if lab_var.get() == 'ARTC':
        plotstrain = plotdf.iloc[1: , 3].astype(float)
        plotstress = plotdf.iloc[1: , 5].astype(float)
     else:
        plotstrain = plotdf['(%)']
        plotstress = plotdf['(MPa)']
    grd = pd.Series(np.gradient(plotstress))
    grd = grd.where(grd>-1).dropna()

    plt.plot(plotstrain[0:len(grd)], plotstress[0:len(grd)])
    plt.legend(labels = names)
    plt.xlabel('Tensile Strain (%)', fontsize=16)
    plt.ylabel('Tensile Stress (MPa)', fontsize=16)
    plt.ylim(ymin=0, ymax = stress_axis_limit)
    plt.xlim(xmin=0, xmax = strain_axis_limit)
    plt.title(str(table_title), fontsize=20)
    plt.savefig('Overlay' + str(file_title))
    plt.close()
    
    
    
    #creates a dataframe of moduli & break/yield properties to write to TAR report-style docx table
    alldata = [stress, stress_dev, strain, strain_dev, moduli, moduli_dev]
    alldata = pd.DataFrame(alldata, index = ['Modulus (MPa)', 'Stdev', stress_indexer, 'Stdev', strain_indexer, 'Stdev'],  columns = names)
#    with pd.ExcelWriter('test.xlsx') as writer:
#        alldata.to_excel(writer, sheet_name = 'Report')
#        writer.sheets['Report'].insert_image('A10', 'Overlay.png')
    
    #creates word file that will contain table
    doc = docx.Document()
    font = doc.styles['Normal'].font
    font.name = 'Arial'
    
    t = doc.add_table(alldata.shape[0]+2, alldata.shape[1]+2)
    
    for i in range(alldata.shape[-1]):
        t.cell(1,i+2).text = str(alldata.columns[i])
    for i in range(alldata.shape[0]):
        for j in range (alldata.shape[-1]):
            t.cell(i+2, j+2).text = str(alldata.values[i,j])
    
    #helper functions to underline rows and columns
    def make_rows_underline(*rows):
        for row in rows:
            for cell in row.cells:
                 for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        run.font.underline = True
    def make_columns_underline(*columns):
        for column in columns:
                for cell in column.cells:
                    for paragraph in cell.paragraphs:
                        for run in paragraph.runs:
                            run.font.underline = True
    
    #merging cells and writing lables
    t.cell(2,1).merge(t.cell(3,1))
    t.cell(4,1).merge(t.cell(5,1))
    t.cell(6,1).merge(t.cell(7,1))
    t.cell(0,1).merge(t.cell(0,int(alldata.shape[1]+1)))
    t.cell(0,0).merge(t.cell(1,0))
    t.cell(3,1).text = 'ITSB'
    t.cell(5,1).text = 'ITEBB'
    t.cell(7,1).text = 'ITMXG'
    t.cell(2,0).text = 'Stress at Yield (MPa)'
    t.cell(3,0).text = 'Stdev'
    t.cell(4,0).text = 'Strain at Yield (MPa)'
    t.cell(5,0).text = 'Stdev'
    t.cell(6,0).text = 'Modulus (MPa)'
    t.cell(7,0).text = 'Stdev'
   
    t.cell(0,1).text = str(table_title)
    t.cell(1,1).text = 'Test Code'
    
    #underlining
    make_rows_underline(t.rows[1])
    make_columns_underline(t.columns[0])
    
    #centers text
    for row in t.rows:
        for cell in row.cells:
            paragraphs = cell.paragraphs
            for paragraph in paragraphs:
                for run in paragraph.runs:
                    font = run.font
                    font.size= Pt(10.5)
                paragraph_format = paragraph.paragraph_format     
                paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    #insert DSM logo
    pic='O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\logo.png'
    paragraph = t.cell(0,0).paragraphs[0]
    run = paragraph.add_run()
    run.add_picture(pic, height=Inches(.50), width=Inches(1.47))
    
    #bold and enlarge table title (grade name)
    title_p = t.cell(0,1).paragraphs[0].runs[0]
    font = title_p.font
    font.size = Pt(18)
    font.bold = True
    
    t.style = 'Table Grid'
    t.font = 'Arial'
    doc.save(str(file_title) + '.docx')

    
    
    
    
    
root = Tk()
root.title('TAR Table Generator')
lab_var = StringVar()
glass_var = StringVar()

rb_frame = Frame(root, borderwidth=1, relief="solid")
rb_frame.grid(row=0, column =0)
glass_rb_frame = Frame(rb_frame,borderwidth=1, relief="solid")
glass_rb_frame.grid(row=0,column=0)

label1 = Label(glass_rb_frame, text = "Would you like to report properties at break or yield?").grid(row=0,column=0)
R1 = Radiobutton(glass_rb_frame, text="Break Properties", variable=glass_var, value='Break Properties',
                       tristatevalue = 0)
R1.grid(row=1,column=0)
R2 = Radiobutton(glass_rb_frame, text="Yield Properties", variable=glass_var, value='Yield Properties',
                       tristatevalue = 0)
R2.grid(row=1,column=1)

lab_label = Label(rb_frame, text = 'Was The Raw Data Generated At Intertek Or ARTC?').grid(row=2, column=0)
R3 = Radiobutton(rb_frame, text="ARTC", variable=lab_var, value='ARTC', 
                       tristatevalue = 0)
R3.grid(row=3,column=0)
R4 = Radiobutton(rb_frame, text="Intertek", variable=lab_var, value='Intertek',
                       tristatevalue = 0)
R4.grid(row=3,column=1)

label2 = Label(root, text = 'Please select a folder to save files in').grid(row=3,column=0)
button1 = Button(root, text = 'Browse', command = browse_folder).grid(row=4,column=0)
folder_label = Label(root)
folder_label.grid(row=5,column=0)


file_label = Label(root, text = 'Browse For Source Files').grid(row=6,column=0)
lb1 = Listbox(root)
lb1.grid(row=7,column=0)
button2 = Button(root, text = 'Select Files', command = select_files).grid(row=8,column=0)

table_title_label = Label(root, text = 'Please Name The Output Table').grid(row=9,column=0)
table_title_entry = Entry(root)
table_title_entry.grid(row=10, column=0)

file_title_label = Label(root, text = 'Please Name The Output File').grid(row=11,column=0)
file_title_entry = Entry(root)
file_title_entry.grid(row=12, column=0)

button3 = Button(root, text = 'Make Table', command = make_table).grid(row=13, column=0)

root.mainloop()

O:/Tech_Service/Analytical - New/Analytical/TAR Data/2019/19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019/python files
